In [1]:
import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np


In [20]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

print(subjects)
# for sub in subjects:

sub='D0059' #why does this not work in the for loop???
# Load the data
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                           extension='.edf', desc='clean', preload=False) #get line-noise filtered data
print(filt)

## Crop raw data to minimize processing time
good = crop_empty_data(filt)

# good.drop_channels(good.info['bads'])
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
# good.info['bads'] += channel_outlier_marker(good, 4, 2)
# good.drop_channels(good.info['bads'])
good.load_data()

ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# Remove intermediates from mem
# good.plot()

## epoching and trial outlier removal

save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

  



['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-02_d

c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (185) does not match the number of channels in the raw data file (184). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (185) does not match the number of channels in the raw data file (184). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (185) does not match the number of channels in the raw data file (184). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0059\ieeg\sub-D0059_acq-01_space-ACPC_electrodes.tsv.


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (185) does not match the number of channels in the raw data file (184). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


<RawEDF | sub-D0059_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 184 x 8769536 (4282.0 s), ~208 kB, data not loaded>
outlier round 1 channels: ['LAIP9']
outlier round 1 channels: ['LAIP9', 'LAIP14']
outlier round 1 channels: ['LAIP9', 'LAIP14', 'LMMT2']
outlier round 1 channels: ['LAIP9', 'LAIP14', 'LMMT2', 'LMMT12']
outlier round 1 channels: ['LAIP9', 'LAIP14', 'LMMT2', 'LMMT12', 'LMPT10']
outlier round 1 channels: ['LAIP9', 'LAIP14', 'LMMT2', 'LMMT12', 'LMPT10', 'LMPT11']
outlier round 2 channels: ['LAIP9', 'LAIP14', 'LMMT2', 'LMMT12', 'LMPT10', 'LMPT11', 'LMPF15']
outlier round 2 channels: ['LAIP9', 'LAIP14', 'LMMT2', 'LMMT12', 'LMPT10', 'LMPT11', 'LMPF15', 'LMMT1']
outlier round 2 channels: ['LAIP9', 'LAIP14', 'LMMT2', 'LMMT12', 'LMPT10', 'LMPT11', 'LMPF15', 'LMMT1', 'LMMT3']
outlier round 2 channels: ['LAIP9', 'LAIP14', 'LMMT2', 'LMMT12', 'LMPT10', 'LMPT11', 'LMPF15', 'LMMT1', 'LMMT3', 'LMMT11']
Reading 0 ... 3242460  =      0.000 ...  1583.232 secs...
Applying average refe

### for a single trial type

In [21]:
# trialType = "i75s25"
trialTypes = ["i75s25", "c75s25", "i75r25", "c75r25", "i75s75", "c75s75", "i75r75", "c75r75",
              "i25s25", "c25s25", "i25r25", "c25r25", "i75s25", "c25s75", "i25r75", "c25r75"]

for trialType in trialTypes:
    for epoch, t, name in zip(
        ("Stimulus"+trialType, "Stimulus"+trialType,  "Response"+trialType),
        ((-0.5, 0), (-0.5, 2), (-1, 2)),
        ("baselineStimulus"+trialType, "Stimulus"+trialType, "Response"+trialType)):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        spec = wavelet_scaleogram(trials, n_jobs=-2, decim=int(
            good.info['sfreq'] / 100))
        crop_pad(spec, "0.5s")
        if name == "baselineStimulus"+trialType:
            base = spec.copy()
            continue
        spec_a = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
        spec_a._data = np.log10(spec_a._data) * 20
        fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
        spec_a.info['subject_info']['files'] = tuple(fnames)
        spec_a.info['bads'] = good.info['bads']
        filename = os.path.join(save_dir, f'{name}-tfr.h5')
        mne.time_frequency.write_tfrs(filename, spec_a, overwrite=True)
        spec_a.save(os.path.join(save_dir, f'{name}-avg.fif'), overwrite=True)


Used Annotations descriptions: ['Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75n25', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25n25', 'Responsei25n75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75n25', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n25', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
18 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 18 events and 3073 original time points ...


0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:    7.0s


Not setting metadata
Used Annotations descriptions: ['Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75n25', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25n25', 'Responsei25n75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75n25', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n25', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
18 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 18 events and 6145 original time points ...


[Parallel(n_jobs=-2)]: Done 174 out of 174 | elapsed:    8.1s finished


...

Applying baseline correction (mode: ratio)
Used Annotations descriptions: ['Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75n25', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25n25', 'Responsei25n75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75n25', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n25', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 6145 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   19.7s


Not setting metadata


[Parallel(n_jobs=-2)]: Done 174 out of 174 | elapsed:   22.6s finished


Applying baseline correction (mode: ratio)


### to average across all trial types and just look at differences between stimulus and response

In [15]:
trialTypes = ["i75s25", "c75s25", "i75r25", "c75r25", "i75s75", "c75s75", "i75r75", "c75r75",
              "i25s25", "c25s25", "i25r25", "c25r25", "i75s25", "c25s75", "i25r75", "c25r75"]

# Define the epoch, time, and name for stimulus and response
stimulus_epoch = "Stimulus"
stimulus_time = (-0.5, 0)
stimulus_name = "Stimulus"
response_epoch = "Response"
response_time = (-0.5, 0.5)
response_name = "Response"

# Initialize empty spectrogram arrays for stimulus and response
stimulus_spec_sum = np.zeros((23, 176, 46, 50))
response_spec_sum = np.zeros((23, 176, 46, 50))

# Loop over trialTypes
for trialType in trialTypes:
    # Define the epoch, time, and name based on trialType
    stimulus_epoch_type = stimulus_epoch + trialType
    response_epoch_type = response_epoch + trialType
    stimulus_name_type = stimulus_name + trialType
    response_name_type = response_name + trialType
    
    # Set the times for stimulus and response
    stimulus_times = [stimulus_time[0] - 0.5, stimulus_time[1] + 0.5]
    response_times = [response_time[0] - 0.5, response_time[1] + 0.5]
    
    # Load the trials for stimulus and response
    stimulus_trials = trial_ieeg(good, stimulus_epoch_type, stimulus_times, preload=True)
    response_trials = trial_ieeg(good, response_epoch_type, response_times, preload=True)
    
    # Remove outliers from trials
    outliers_to_nan(stimulus_trials, outliers=10)
    outliers_to_nan(response_trials, outliers=10)
    
    # Compute spectrograms for stimulus and response
    stimulus_spec = wavelet_scaleogram(stimulus_trials, n_jobs=-2, decim=int(good.info['sfreq'] / 100))
    response_spec = wavelet_scaleogram(response_trials, n_jobs=-2, decim=int(good.info['sfreq'] / 100))
    
    # Crop and pad spectrograms
    crop_pad(stimulus_spec, "0.5s")
    crop_pad(response_spec, "0.5s")
    
    # Initialize base spectrogram for stimulus
    if stimulus_spec_sum is None:
        stimulus_spec_sum = stimulus_spec.copy()
    else:
        # Add stimulus spectrogram to the sum
        stimulus_spec_sum._data += stimulus_spec._data
    
    # Initialize base spectrogram for response
    if response_spec_sum is None:
        response_spec_sum = response_spec.copy()
    else:
        # Add response spectrogram to the sum
        response_spec_sum._data += response_spec._data

# Compute the average spectrogram for stimulus and response
stimulus_spec_avg = stimulus_spec_sum / len(trialTypes)
response_spec_avg = response_spec_sum / len(trialTypes)

# Transform spectrogram data to decibels
stimulus_spec_avg._data = np.log10(stimulus_spec_avg._data) * 20
response_spec_avg._data = np.log10(response_spec_avg._data) * 20

# Update file information and bad channels
fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
stimulus_spec_avg.info['subject_info']['files'] = tuple(fnames)
response_spec_avg.info['subject_info']['files'] = tuple(fnames)
stimulus_spec_avg.info['bads'] = good.info['bads']
response_spec_avg.info['bads'] = good.info['bads']

# Define the save directory
save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the average spectrograms with the desired filename format
stimulus_filename = os.path.join(save_dir, f'{stimulus_name}-allTrialTypesAvgtfr.h5')
response_filename = os.path.join(save_dir, f'{response_name}-allTrialTypesAvgtfr.h5')
mne.time_frequency.write_tfrs(stimulus_filename, stimulus_spec_avg, overwrite=True)
mne.time_frequency.write_tfrs(response_filename, response_spec_avg, overwrite=True)

# Save the average spectrograms in the -avg.fif format
stimulus_avg_filename = os.path.join(save_dir, f'stimulus-allTrialTypesAvg.fif')
response_avg_filename = os.path.join(save_dir, f'response-allTrialTypesAvg.fif')
stimulus_spec_avg.save(stimulus_avg_filename, overwrite=True)
response_spec_avg.save(response_avg_filename, overwrite=True)

Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 3073 original time points ...


0 bad epochs dropped
Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
23 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 23 events and 4097 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   10.0s


Not setting metadata


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   11.9s finished
[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   13.9s


Not setting metadata


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   16.2s finished


AttributeError: 'numpy.ndarray' object has no attribute '_data'